<img src = './soai_logo.png' width = 10% align = right>

# Modelo usando `sklearn`
Nesse script introdutório nós vamos:
* Importar os dados
* Motivação, visualização e motivação (simples)
* Implementação de modelos usando `sklearn`
* Comparação de performance dos modelos
* Análise do melhor modelo

<img src = './iris_with_labels.jpg' width = 40% align = right>

## Dataset: Iris - 'Hello World'

O dataset contém 3 espécies diferentes de íris:
* Setosa
* Versicolour
* Virginica

Para cada tipo flor examinada, foram medidas:
* Sepal witdh (largura)
* Sepal length (comprimento)
* Petal witdh (largura)
* Petal length (comprimento)

[Mais informações.](https://en.wikipedia.org/wiki/Iris_flower_data_set)



**Importando as bibliotecas e dados**

In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['species'] = iris.target
iris_df['species'] = iris_df['species'].apply(lambda x: iris.target_names[x])
iris_df.head()

**Análise exploratória de dados**

In [ ]:
iris_df.describe(include='all')

In [ ]:
iris_df['species'].value_counts(normalize=False)

**Visualização dos dados**

In [ ]:
plt.figure(figsize=(8, 6), dpi=80)
sns.scatterplot(x='sepal length (cm)', y='sepal width (cm)', hue='species', data=iris_df)
plt.title('Sepal width vs. length')
plt.grid()
plt.show()

Redução dimensional usando [PCA](https://pt.wikipedia.org/wiki/An%C3%A1lise_de_componentes_principais) para visualização em 3 dimensões.

In [ ]:
iris_reduced = pd.DataFrame(data=PCA(n_components=3).fit_transform(iris.data),
                            columns=['1st PC','2nd PC','3rd PC'])
iris_reduced['species'] = iris_df['species']

fig = px.scatter_3d(iris_reduced, x='1st PC',y='2nd PC',z='3rd PC', color='species')
fig.show()

**Preparando os dados para treinar os modelos**

In [ ]:
seed = 10
X_train, X_test, y_train, y_test = train_test_split(iris_df.iloc[:,:4],
                                                    iris_df.iloc[:,4], test_size=0.2,
                                                    random_state=seed)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
results = {}

**Treinando os modelos**

* Regressão Logística
* Naive Bayes
* Árvores de decisão
* KNN
* SVM (Linear, RBF)
* Random Forest

Também vamos implementar um k-Fold crossvalidation ([validação cruzada](https://pt.wikipedia.org/wiki/Valida%C3%A7%C3%A3o_cruzada)) para selecionar o modelo. Com `sklearn` o processo é muito similar quando se tem os dados preparados. Os resultados são registrados no dicionário `results`.

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
CV_res = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')
results['LR'] = CV_res

Para evitar o processo repetitivo do código acima, podemos usar um `for-loop` para os demais modelos.

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
models = [('NB', GaussianNB()),
          ('DT', DecisionTreeClassifier()),
          ('KNN', KNeighborsClassifier()),
          ('linSVC', LinearSVC()),
          ('rbfSVC', SVC()),
          ('RF', RandomForestClassifier())]
for name, model in models:
    CV_res = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')
    results[name] = CV_res

In [ ]:
print('Acurácia/exatidão dos modelos (desvio padrão):')
for model in results.keys():
    print('{}: {:.3} ({:.3})'.format(model, results[model].mean(), results[model].std()))

**Vamos selecionar a `LogisticRegression` para avaliar a performance**
Avaliação será feita em termos de [precisão](https://pt.wikipedia.org/wiki/Precis%C3%A3o_e_revoca%C3%A7%C3%A3o), [revocação](https://pt.wikipedia.org/wiki/Precis%C3%A3o_e_revoca%C3%A7%C3%A3o), [F1](https://pt.wikipedia.org/wiki/Precis%C3%A3o_e_revoca%C3%A7%C3%A3o) e [matriz de confusão](https://en.wikipedia.org/wiki/Confusion_matrix).

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

clf = LogisticRegression()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
report = classification_report(y_test, predictions, output_dict=True)
sns.heatmap(confusion_matrix(y_test, predictions), annot=True,annot_kws={"size": 16})
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()
pd.DataFrame(data=report).T

## Mais informações:
* [Principal Component Analysis - Explained Visually](http://setosa.io/ev/principal-component-analysis/)
* [Entendendo o que é Matriz de Confusão com Python](https://medium.com/data-hackers/entendendo-o-que-%C3%A9-matriz-de-confus%C3%A3o-com-python-114e683ec509)